In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.learning_curve import learning_curve
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import confusion_matrix

# Exercise 1

## Importing and simple cleaning of data

First of all, some simple cleaning. We remove features that have nothing to do with the skin color. We remove player (we will use 'playerShort' later for aggregating), birthday, Alpha_3 (since it is the same as refCountry) and photoID.

There are missing values for height, weight and position. And also 163 dyads miss the information for the implicit association test and the explicit bias scores.

First we will remove the referees that got in the data by mistake (as mentioned in the preprocessing article) and see if the problem is solved then. After removing these referees, there were still 110 samples with missing data which we just removed.

The missing values of height and weight are replaced by the respective means and the missing values in position are replaced by the most frequently occuring position.

In [ ]:
df = pd.read_csv ('CrowdstormingDataJuly1st.csv')
df.head ()

In [ ]:
# remove values without skin color rating (we know all samples have either two raters or none)
df_train_raw = df [pd.notnull (df ['rater1'])]

In [ ]:
# drop unimportant features
df_train_raw = df_train_raw.drop(['birthday', 'player', 'Alpha_3', 'photoID'], axis=1)

In [ ]:
# remove referees that are not supposed to be in here
referees_to_remove = df_train_raw.groupby('refNum').sum()[df_train_raw.groupby('refNum').sum()['games'] < 22].index.tolist()
df_train_raw = df_train_raw[~df_train_raw.refNum.isin(referees_to_remove)]

In [ ]:
# remove samples that don't have IAT or Exp score information
df_train_raw = df_train_raw[df_train_raw.meanIAT.notnull()]

In [ ]:
# set missing height, weight and position values to mean
mean_height = df_train_raw.height.mean()
mean_weight = df_train_raw.weight.mean()
most_frequent_position = df_train_raw['position'].value_counts().index[0]

df_train_raw.loc[df_train_raw.height.isnull(),'height'] = mean_height
df_train_raw.loc[df_train_raw.weight.isnull(),'weight'] = mean_weight
df_train_raw['position'] = df_train_raw['position'].fillna(most_frequent_position)

## Advanced pre-processing

We create a new feature called "skinColor" which is the average of both ratings, then mapped to either "white" or "black" to simplify our classification.

Position, club and leagueCountry are not numerical, we use dummy variables to make them numerical. For now we left out the club feature because it would cause a lot of dummy variables and we assume it is not a significant feature.

In [ ]:
df_train = df_train_raw

# create feature "skinColor"
def attribute_skin_label(val):
    if val > 0.5:
        return 1 #  "black"
    else:
        return 0  # "white"
    
#Replace rating with either white or black
df_train['skinColor'] = df_train[['rater1','rater2']].mean(axis=1).apply(attribute_skin_label)
df_train = df_train.drop(['rater1', 'rater2'], 1)

# add dummy variables for position and leagueCountry
n_positions = len(df_train.position.unique())
n_leagueCountry = len(df_train.leagueCountry.unique())

d_positions = pd.get_dummies(df_train['position'])
d_leagueCountry = pd.get_dummies(df_train['leagueCountry'])

df_train = pd.concat([df_train, d_positions, d_leagueCountry], axis=1)
df_train = df_train.drop(['position', 'club', 'leagueCountry'], 1)

In [ ]:
# Checking how many "white" and "black" players we have
df_train['skinColor'].value_counts()

## Player aggregation and feature engineering
We added 6 new features: the meanIAT and the meanExp per dyad are multiplied with each of the values 'redCards', 'YellowCards', and 'YellowRedCards'. We do this because we assume that in a specific encounter between a referee and a player, the combination of a given card and the bias score of the referee's country, is a large indicator for darker skin color.

We aggregate per player because we want to predict the skin color of a given player, not of a given dyad. To obtain an intuitive result, we choose to sum over values such as games, games outcomes and cards and mean over the dummy variables and bias scores.

In [ ]:
df_train['IATRedCards'] = df_train['redCards']*df_train['meanIAT']
df_train['ExpRedCards'] = df_train['redCards']*df_train['meanExp']
df_train['IATYellowRedCards'] = df_train['yellowReds']*df_train['meanIAT']
df_train['ExpYellowRedCards'] = df_train['yellowReds']*df_train['meanExp']
df_train['IATYellowCards'] = df_train['yellowCards']*df_train['meanIAT']
df_train['ExpYellowCards'] = df_train['yellowCards']*df_train['meanExp']

In [ ]:
# aggregation per player using 'playerShort'
summed = df_train[['playerShort','games', 'victories', 'ties', 'defeats', 'goals', 'yellowCards', 'yellowReds', 'redCards']].groupby('playerShort').sum()
meaned = df_train[['playerShort', 'skinColor','Attacking Midfielder', 'Center Back', 'Center Forward',
       'Center Midfielder', 'Defensive Midfielder', 'Goalkeeper',
       'Left Fullback', 'Left Midfielder', 'Left Winger', 'Right Fullback',
       'Right Midfielder', 'Right Winger', 'England', 'France', 'Germany',
       'Spain', 'meanIAT', 'meanExp', 'IATRedCards', 'ExpRedCards', 'IATYellowRedCards', 'ExpYellowRedCards', 'IATYellowCards', 'ExpYellowCards', 'weight', 'height' ]].groupby('playerShort').mean()

In [ ]:
df_agg = pd.concat([summed, meaned], axis=1)

In [ ]:
df_agg.head()

In [ ]:
df_agg.columns

## Making the model


### Helper functions

In [ ]:
# Helper function for displaying feature importance
def feature_importance(X, features, n_most_important_features=5):
    """
    Prints the n most important features of the classifier in order
    Format: (feature_name, importance)
    @param X: The data matrix
    @param features: clf.feature_importances_
    @param n_most_important_features
    """
    feature_importance = {}
    for x,y in zip(X.columns,features):
        feature_importance[x]=y
    import operator
    sorted_feature_importance = sorted(feature_importance.items(), key=operator.itemgetter(1),reverse=True)
    for i in range(min(len(features),n_most_important_features)):
        print (sorted_feature_importance[i])
    
    return sorted_feature_importance

In [ ]:
# Helper function for plotting learning curves
def plot_learning_curve(title, estimator, X, y, cv=30):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    title : string
        Title for the chart.
        
    estimator: clf

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
    """
    plt.figure()
    plt.title(title)

    plt.xlabel("Training examples")
    plt.ylabel("Score")
    
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, cv=cv, n_jobs=-1)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

### Model 1: Baseline model, only based on cards

We start off with a simple model.

In [ ]:
clf = RandomForestClassifier(n_estimators=500, max_features='log2', max_depth=8, min_samples_leaf=2)
Y = np.asarray(df_agg['skinColor'], dtype='str')
X = df_agg[['yellowCards','yellowReds', 'redCards']]

In [ ]:
X.columns  # Features in model

In [ ]:
print ('Cross validation scores:')
print(cross_val_score(clf, X, Y, cv=20, scoring='accuracy'))
clf.fit(X,Y)
print('Features ranked by importance:')
feature_importance(X, clf.feature_importances_)

In [ ]:
title = "Learning Curves (RandomForestClassifier) - Model 1"

plot_learning_curve(title, clf, X, Y)

plt.show()

### Model 2: Model with all raw data that came out of preprocessing (no combined features)

We decide to keep meanIAT and meanExp. As we have aggregated by player, they represent the average amount of bias the players receive over all the referees they meet. 

In [ ]:
clf = RandomForestClassifier(n_estimators=500, max_features='log2', max_depth=8, min_samples_leaf=2)
Y = np.asarray(df_agg['skinColor'], dtype='str')
X = df_agg.drop(['skinColor','IATRedCards','ExpRedCards', 'IATYellowRedCards', 'ExpYellowRedCards','IATYellowCards', 'ExpYellowCards' ], 1)

In [ ]:
X.columns  # Features in model

In [ ]:
print ('Cross validation scores:')
print(cross_val_score(clf, X, Y, cv=20, scoring='accuracy'))
clf.fit(X,Y)
print('Features ranked by importance:')
dropping_order = feature_importance(X, clf.feature_importances_, 7)

In [ ]:
title = "Learning Curves (RandomForestClassifier) - Model 2"

plot_learning_curve(title, clf, X, Y)

plt.show()

### Model 3: Model with added features that multiply the meanIAT and the meanExp with the red/yellow/yellowred cards values

In [ ]:
clf = RandomForestClassifier(n_estimators=500, max_features='log2', max_depth=8, min_samples_leaf=2)
Y = np.asarray(df_agg['skinColor'], dtype='str')
X = df_agg.drop(['skinColor'], 1)

In [ ]:
X.columns  # Features in model

In [ ]:
print ('Cross validation scores:')
print(cross_val_score(clf, X, Y, cv=20, scoring='accuracy'))
clf.fit(X,Y)
print('Features ranked by importance:')
feature_importance(X, clf.feature_importances_, 7)

In [ ]:
title = "Learning Curves (RandomForestClassifier) - Model 3"

plot_learning_curve(title, clf, X, Y)

plt.show()

### Conclusions:

By simply taking the cards as input data, we are able to make a pretty good prediction as to the skin color of the player (Model 1). This could lead to the conclusion that racial bias is a significant factor. The learning graph shows that the variance in model performance is very large for different folds in the cross-validation.

By including data on the the racial biases of the referees (depending on the country they come from), we are able to improve the model by about 2 percentage points. But more importantly the model exhibits a significantly lower variance. This seems reasonable because these values show the way referees give out the cards differently to people of different color.

Concerning the feature importance we see that in Model 1 that yellow cards have the greatest importance. Since giving a red card is a very strong decision it will be influenced less by racial bias. Giving a yellow card is a softer decision, thus leaving room for objectivity and influence. We can observe this by inspecting the feature importance. In Model 1 the most important feature is by far the amount of YellowCards received. In Model 2 the bias scores are the biggest influencers.

This leads us to build the 3th model using a combination of the number of cards received and the bias scores. Our assumption is confirmed because the combination of bias scores with received yellow card are observed as important features. Although Model 3 does not score much higher, this leads to the conclusion that the extra features do not offer more added value than the average of meanIAT or meanExp over all refrees for a given player.

# Exercise 2

### KMeans clustering while dropping features one by one

We take the dropping order based on the reversed feature importance result given by Model 2.

In [ ]:
drop_list = [x[0] for x in dropping_order]
drop_list = drop_list[::-1]
drop_list

In [ ]:
def silhouette_score_drop(df, c_drop, n_clust):
    while len(c_drop) > 0:
        kmeans = KMeans(n_clusters=n_clust, init='k-means++').fit(df[c_drop])
        print("- Mixed percentage: ", abs(df['skinColor'] - kmeans.labels_).sum()/df.shape[0])
        print("- Silhouette score: ", silhouette_score(df[c_drop], kmeans.labels_, metric='euclidean'))
        print("- Confusion matrix: \n", confusion_matrix(df['skinColor'], kmeans.labels_))
        print("------------------------------------------------")
        c_drop = c_drop[1:]

In [ ]:
silhouette_score_drop(df_agg, drop_list, 2)

In [ ]:
silhouette_score_drop(df_agg, drop_list, 4)

The above iterative dropping of features gives us a clustering in 2 groups with a silhouette score 0.9 where playes with dark or light skin are mixed. Being a player from France has a bigger influence on the clustering than the bias scores.

In [ ]:
kmeans = KMeans(n_clusters=2, init='k-means++').fit(df_agg[['France','meanIAT','meanExp']])
print("- Mixed percentage: ", abs(df_agg['skinColor'] - kmeans.labels_).sum()/df_agg.shape[0])
print("- Silhouette score: ", silhouette_score(df_agg[['France','meanIAT','meanExp']], kmeans.labels_, metric='euclidean'))
print("- Confusion matrix: \n", confusion_matrix(df_agg['skinColor'], kmeans.labels_))

Obviously we can find a perfect silhouette score based on the team of theplayer where the players with different skin color are very mixed. But this feature is not a feature of the referee.

In [ ]:
kmeans = KMeans(n_clusters=4, init='k-means++').fit(df_agg[['France','Germany','England','Spain']])
print("- Mixed percentage: ", abs(df_agg['skinColor'] - kmeans.labels_).sum()/df_agg.shape[0])
print("- Silhouette score: ", silhouette_score(df_agg[['France','Germany','England','Spain']], kmeans.labels_, metric='euclidean'))
print("- Confusion matrix: \n", confusion_matrix(df_agg['skinColor'], kmeans.labels_))